In [1]:
from sklearn.datasets import load_boston
import numpy as np
import keras
from keras import models, layers
from keras.callbacks import EarlyStopping
import tensorflow as tf

d = load_boston()

Using TensorFlow backend.


In [2]:
d.data.shape

(506, 13)

In [117]:
data = d.data
temp_data = (data - np.mean(data))/np.std(data)
target = d.target
target = (target - np.mean(target))/np.std(target)

In [118]:
data = []
lags = 10

for i in range(lags, len(temp_data)):
    temp = temp_data[max(0, i-lags):i]
    data.append(temp)
    
data = np.array(data)

In [120]:
data[0]

array([[-0.48270736, -0.35874599, -0.46683694, -0.4827509 , -0.47904453,
        -0.43745466, -0.03357755, -0.45457423, -0.47586174,  1.55644101,
        -0.37734672,  2.25155744, -0.44844287],
       [-0.48256276, -0.4827509 , -0.43404453, -0.4827509 , -0.47951988,
        -0.43851559,  0.06080397, -0.44853174, -0.46897258,  1.18442627,
        -0.36012382,  2.25155744, -0.41978396],
       [-0.4825629 , -0.4827509 , -0.43404453, -0.4827509 , -0.47951988,
        -0.43325227, -0.06182311, -0.44853174, -0.46897258,  1.18442627,
        -0.36012382,  2.22351855, -0.45498758],
       [-0.4825279 , -0.4827509 , -0.46773253, -0.4827509 , -0.47959566,
        -0.43454055, -0.16722729, -0.44098742, -0.46208341,  1.04664303,
        -0.35392357,  2.23591904, -0.46249676],
       [-0.4822752 , -0.4827509 , -0.46773253, -0.4827509 , -0.47959566,
        -0.43351406, -0.10935833, -0.44098742, -0.46208341,  1.04664303,
        -0.35392357,  2.25155744, -0.44603167],
       [-0.48254526, -0.482750

In [ ]:
np.random.seed(0)
w = [0] * 3
w[0] = np.random.normal(0, 0.1, (13,2))
w[1] = np.random.normal(0, 0.1, 2)
w[2] = np.random.normal(0, 0.1, 2)

In [152]:
def update(x, y, h, w, lr=0.1):
    w_xh = np.zeros_like(w[0])
    w_hh = np.zeros_like(w[1])
    w_hy = np.zeros_like(w[2])
    loss = 0
    for i in range(len(x)):
        for j in range(len(x[0])-1): 
            y_hat = np.tanh(x[i][j]@w[0] + h@w[1]) * w[2]
            print(y_hat)
            print(x[i][j])
            np.add(w_xh, x[i][j]* np.tanh(x[i][j]@w[0] + h@w[1])*w[2] * (x[i][j+1]-y_hat))
#             w_xh += x[i][j]* np.tanh(x[i][j]@w[0] + h@w[1])*w[2] * (x[i][j+1]-y_hat)
            w_hh += h * np.tanh(x[i][j]@w[0] + h*w[1])*w[2] * (x[i][j+1]-y_hat) ## 평가지표를 뭐로 잡고 평가해야되는가?
            w_hy += np.tanh(x[i][j]@w[0] + h*w[1]) * (x[i][j+1]-y_hat)
            h = x[i][j] @ w[0]
        w[0] += w_xh *lr/len(x[i])
        w[1] += w_hh *lr/len(x[i])
        w[2] += w_hy *lr/len(x[i])
        h = x[i][-1] @ w[0]
        y_hat = np.tanh(x[i][-1] @ w[0] + h@w[1]) * w[2]
        loss += (y[i] - y_hat) ** 2
        
    return w, h, loss
        
        
    
    
    

In [153]:
np.random.seed(0)
w = [0] * 3
w[0] = np.random.normal(0, 0.1, 13)
w[1] = np.random.normal(0, 0.1, 1)
w[2] = np.random.normal(0, 0.1, 1)

h = np.random.normal(0, 0.1, 1)
hist = []

for i in range(100):
    w, h, c = update(data, target, h, w, lr = 0.01)
    hist.append(c)
    

[0.00269089]
[-0.48270736 -0.35874599 -0.46683694 -0.4827509  -0.47904453 -0.43745466
 -0.03357755 -0.45457423 -0.47586174  1.55644101 -0.37734672  2.25155744
 -0.44844287]


ValueError: non-broadcastable output operand with shape (1,) doesn't match the broadcast shape (13,)

In [114]:
## 실패작
np.random.seed(0)
w = [0] * 3
w[0] = np.random.normal(0, 0.1, 13)
w[1] = np.random.normal(0, 0.1, 1)
w[2] = np.random.normal(0, 0.1, 1)

h = np.random.normal(0, 0.1, 1)
hist = []
def update(x, y, h, w, lr=0.01):
    z = np.array(list(map(np.tanh, x @ w[0] + h @ w[1])))
    
    y_hat = z * w[2]
    
    loss = (y - y_hat).T @ (y - y_hat)
    

    w_hy2 = z.T @ (y - y_hat)
    
    w_xh = (x.T @ ((1-z)*(1+z))).reshape((13, 1)) @ np.repeat(w_hy, len(x)).reshape((1,len(x))) @ (y-y_hat)
    
    w_hh = (np.repeat(h, len(x)).reshape(1,len(x)) @ ((1-z)*(1+z))) @ np.repeat(w_hy, len(x)).reshape((1,len(x))) @ (y-y_hat)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
    w[0] += w_hy2 * lr/len(x)
    w[1] += w_hh * lr/len(x)
    w[2] += w_hy2 * lr/len(x)
    
    return w, loss
    

In [115]:
np.random.seed(0)
w = [0] * 3
w[0] = np.random.normal(0, 0.1, 13)
w[1] = np.random.normal(0, 0.1, 1)
w[2] = np.random.normal(0, 0.1, 1)

h = np.random.normal(0, 0.1, 1)
hist = []

w_xh = np.ones_like(w[0])
w_hh = np.ones_like(w[1])
w_hy = np.ones_like(w[2])

for _ in range(100):
    w, loss = update(data, target, h, w, lr=0.1)
    print(loss)

506.2586347036717
506.2431039872506
506.2285966972726
506.2150378687755
506.20235851090246
506.19049510219867
506.17938913023016
506.1689866715205
506.159238008171
506.1500972778686
506.1415221542909
506.13347355518454
506.12591537565487
506.11881424441873
506.11213930098495
506.105861991906
506.0999558844183
506.09439649593753
506.0891611380111
506.0842287734656
506.07957988558695
506.0751963582877
506.0710613663002
506.0671592745259
506.0634755457476
506.0599966559776
506.05671001678706
506.0536039040134
506.0506673922968
506.0478902949507
506.0452631087067
506.04277696292115
506.0404235728644
506.0381951967458
506.0360845961604
506.0340849996702
506.032190069254
506.0303938693898
506.0286908385476
506.0270757628942
506.0255437520263
506.0240902165644
506.0227108474575
506.02140159685274
506.02015866040927
506.01897846093635
506.0178576332461
506.01679301012854
506.0157816093559
506.0148206216333
506.0139073994239
506.01303944657747
506.01221440869955
506.01143006420403
506.010684315